<a href="https://colab.research.google.com/github/Kos-Oleg/ttt/blob/main/%D0%9A%D0%BE%D1%81%D1%8B%D1%85_%D0%9E_%D0%92_%D0%A3%D0%91%D0%92%D0%A22301.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
ttt = pd.read_csv('/content/drive/MyDrive/Laptop_price.csv') #Чтение CSV файла

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = '/content/drive/MyDrive/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor),
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')


['/content/drive/MyDrive/laptop_price_model.pkl']

In [53]:
!git init
!git add Laptop_price.csv
!git commit - m "first commit"
!git branch -M main
!git remote add origin https://github.com/Kos-Oleg/ttt.git
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
fatal: pathspec 'Laptop_price.csv' did not match any files
error: pathspec '-' did not match any file(s) known to git
error: pathspec 'm' did not match any file(s) known to git
error: pathspec 'first commit' did not match any file(s) known to git
error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Kos-Oleg/ttt.git'


fatal: pathspec 'Laptop_price.csv' did not match any files


In [ ]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "/content/drive/MyDrive/laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}


Writing app.py


In [ ]:
!pip install python-multipart

In [ ]:
!ngrok config add-authtoken 2uYBVIJ6qWzClp2c0g1pcBg82FS_6AyeEDRHVmN8w2hngYJvw

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &


In [ ]:
from pyngrok import ngrok # Подключаем публичный URL
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)


API доступно по адресу: NgrokTunnel: "https://8b0d-34-48-140-251.ngrok-free.app" -> "http://localhost:8000"
